In [23]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Extract Reduced Frames from Videos
def extract_reduced_frames(video_dir, output_dir, interval=10):
    """
    Extract frames from videos at a specific interval to reduce dataset size.
    :param video_dir: Path to the directory containing videos.
    :param output_dir: Path to the directory to save the extracted frames.
    :param interval: Number of seconds between each extracted frame.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)
        if not video_file.endswith(('.mp4', '.avi', '.mov', '.mkv')):
            continue
        
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second of the video
        frame_interval = int(fps * interval)  # Calculate frame interval in terms of frame count
        
        frame_count = 0
        success, frame = cap.read()
        while success:
            if frame_count % frame_interval == 0:  # Extract frame at the given interval
                frame_filename = os.path.join(output_dir, f"{os.path.splitext(video_file)[0]}_frame_{frame_count}.jpg")
                cv2.imwrite(frame_filename, frame)
            success, frame = cap.read()
            frame_count += 1
        
        cap.release()
    print(f"Reduced frames extracted and saved in {output_dir}")

# Define paths
real_videos_path = r'C:\Users\yakup\Desktop\Special Dataset\DFD_original sequences'
fake_videos_path = r'C:\Users\yakup\Desktop\Special Dataset\DFD_manipulated_sequences'

real_frames_path = r'C:\Users\yakup\Desktop\Special Dataset\Frames\Real'
fake_frames_path = r'C:\Users\yakup\Desktop\Special Dataset\Frames\Fake'

# Extract frames with an interval of 5 seconds
extract_reduced_frames(real_videos_path, real_frames_path, interval=5)
extract_reduced_frames(fake_videos_path, fake_frames_path, interval=5)

# Step 2: Dataset Preparation
frame_dataset_path = r'C:\Users\yakup\Desktop\Special Dataset\Frames'



Reduced frames extracted and saved in C:\Users\yakup\Desktop\Special Dataset\Frames\Real
Reduced frames extracted and saved in C:\Users\yakup\Desktop\Special Dataset\Frames\Fake


In [26]:
# Data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Load dataset
dataset = datasets.ImageFolder(root=frame_dataset_path, transform=transform)

# Split into training and validation sets
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

def compute_flattened_size(model, input_shape):
    with torch.no_grad():
        dummy_input = torch.zeros(1, *input_shape)  # Example input (1 batch, C, H, W)
        output = model.conv_layers(dummy_input)
        return output.view(-1).shape[0]

# Step 3: Define CNN Model
class CNNModel(nn.Module):
    def __init__(self, input_shape=(3, 112, 112)):
        super(CNNModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        flattened_size = compute_flattened_size(self, input_shape)
        self.fc_layers = nn.Sequential(
            nn.Linear(flattened_size, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_layers(x)
        return x


model = CNNModel().to('cuda' if torch.cuda.is_available() else 'cpu')

# Step 4: Training the Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.float().to(device)

        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x86528 and 18432x128)

In [ ]:
# Save the model as .pth
torch.save(model.state_dict(), 'deepfake_detector_model.pth')

In [ ]:
# Step 5: Evaluate the Model
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs).squeeze()
        preds = (outputs > 0.5).int()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=['Real', 'Fake']))

# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()